# AD06 - Dask e NumPy (Comparação, Definição de Chunk)

Vamos criar uma arranjo 2D bastante grande com valores aleatórios que seguem uma distribuição normal, e obter a média dos valores ao longo de um dos eixos de maneira que tenhamos uma visão simplificada dos valores através de um arranjo 1D.

In [1]:
import numpy as np
import dask.array as da

## Versão com NumPy

In [2]:
%%time
xn = np.random.normal(10, 0.1, size=(30_000, 30_000))
yn = xn.mean(axis = 0)
yn

CPU times: user 42.5 s, sys: 3.06 s, total: 45.6 s
Wall time: 47.2 s


array([10.00141012, 10.00049269,  9.99872554, ...,  9.99940139,
       10.00056919, 10.00003686])

## Versão com Dask Array

In [3]:
xd = da.random.normal(10, 0.1, size=(30_000, 30_000), chunks=(3_000, 3_000))
xd

dask.array<normal, shape=(30000, 30000), dtype=float64, chunksize=(3000, 3000), chunktype=numpy.ndarray>

In [4]:
xd.nbytes / 1e9

7.2

In [5]:
yd = xd.mean(axis = 0)
yd

dask.array<mean_agg-aggregate, shape=(30000,), dtype=float64, chunksize=(3000,), chunktype=numpy.ndarray>

In [6]:
%%time
yd.compute()

CPU times: user 57 s, sys: 146 ms, total: 57.1 s
Wall time: 29.4 s


array([10.00036974,  9.99951129, 10.00081276, ...,  9.99999538,
       10.00056352,  9.99977337])

### Questões para discussão
- O que acontece se `chunks = (10_000, 10_000)`?
- O que acontece se `chunks = (30, 30)`?

# Escolher um bom tamanho para cada bloco

A primeira dica para escolher um bom tamanho é observar a escolha.

Vejamos a opção `chunk='auto'` (padrão) de Dask escolhe os tamanho.

Mais sobre definição automática de *chunks*, na URL abaixo:
- https://docs.dask.org/en/stable/array-chunks.html#automatic-chunking

In [7]:
darr = da.random.random((1000, 1000, 1000))
darr

dask.array<random_sample, shape=(1000, 1000, 1000), dtype=float64, chunksize=(250, 250, 250), chunktype=numpy.ndarray>

Vamos alterar o formato dos blocks, com `rechunk`.

In [8]:
darr = darr.rechunk({0: -1, 1: 100, 2: "auto"})
darr

dask.array<rechunk-merge, shape=(1000, 1000, 1000), dtype=float64, chunksize=(1000, 100, 125), chunktype=numpy.ndarray>

## Muito pequeno é problemático

Se cada bloco for pequeno demais, o custo do seu gerenciamento não o justifica. Um bloco deve ter um tamanho que permita um cálculo na ordem de alguns segundos.

Vejamos um caso ruim com uma quantidade excessiva de chunks.

In [9]:
darr = darr.rechunk({0: 10, 1: 10, 2: 10})
darr

,Array,Chunk
Bytes,7.45 GiB,7.81 kiB
Shape,"(1000, 1000, 1000)","(10, 10, 10)"
Count,2040528 Tasks,1000000 Chunks
Type,float64,numpy.ndarray


## Muito grande também é problemático

Se cada bloco for grande demais, existem também problemas.
1. O bloco não cabe na memória (funciona, mas usa o disco durante o cálculo, ineficiente
2. Limitação de paralelismo

O dashboard pode indicar se o tamanho do `chunk` está excessivamente grande.

Vejamos um caso ruim com uma quantidade excessiva de chunks.

In [10]:
darr = darr.rechunk({0: 1000, 1: 1000, 2: 1000})
darr

dask.array<rechunk-merge, shape=(1000, 1000, 1000), dtype=float64, chunksize=(1000, 1000, 1000), chunktype=numpy.ndarray>

## Regras gerais para definir um bom tamanho

1. Recomendação de chunk entre 100MBytes e 1GBytes
    - Salvo claro para casos onde os dados são realmente gigantes
2. Evitar grafos de tarefas muito grandes
3. Ter pelo menos uma quantidade de chunks igual a de cores
4. O custo de execução da tarefa deve ser bem maior que se custo de gerenciamento
5. Blocos definidos devem estar alinhados com os blocos no disco
    - Usar formatos de dados em blocos: HDF5, NetCDF, TIFF, Zarr
    